# Example on how to set up mock API requests

In this notebook you will work through an example of how to build the necessary setup files so you can mock an API request using `mock-request`. In particular, you will store one response and its request parameters, as well as responses for a couple of errored responses.

Here's the breakdown of the steps we'll follow:
1. make a request to the Yelp API
1. pickle the response 
1. save the request parameters and the pickle destination into a JSON file
1. make errored requests and pickle the responses
1. associate those errored responses with error codes (404, etc.), and pair them in a CSV file so that `mock-request` returns those responses when the same errors are found
1. test out your setup by mocking the original request

_Note: In addition to serving the purpose of creating an example, this notebook also creates files that can be used as unit tests._

## 1. Make a request to the Yelp API

In [1]:
import requests
import pickle
import yaml
import pandas as pd

In [2]:
# To save the requests information
from mock_request.utils import save_requests_info

You must create the `./.private/api_key.yml` YAML file with the following structure:

```
- API_KEY:
    <YOUR API KEY VALUE>
```

_(Alternatively, while not recommended, you can just write your API key into `API_KEY`)._

In [3]:
# Read API key
with open("./.private/api_key.yml", "r") as f:
    API_KEY = yaml.load(f)[0]["API_KEY"]

In [4]:
# Define the base URL for the request
base_url = "https://api.yelp.com/v3/businesses/search"

# Set up the request headers -- API key is used here
headers = {"Authorization": "Bearer " + API_KEY}

In [5]:
# Define the request parameters
params = {
    "location": "Newark, NJ",
    "term": "laundromat",
    "limit": 5
}

In [6]:
# Make the request
response = requests.get(
    base_url,
    headers=headers,
    params=params
)

## 2. Pickle the obtained response

Great, now that we have the response it looks like we're ready to save it to disk. But watch out! The response contains the request, which contains the API key (you can see this by exploring the `response.request.headers` attribute). Since we don't want an exposed key in the pickle object we may share with others or have in production code, let's delete it from the response before before saving it.

In [7]:
def pickle_response(response, path, auth_header="Authorization"):
    """Save a response object as a pickle file.
    
    The API key authentication header will be deleted if the proper
    
    :param response: A response object containing an API request and 
        response.
    :param path: Path to file for pickled response.
    :param auth_header: Key to the value in the API request headers
        holding the API key (if any). Defaults to "Authorization".
    
    """

    if  auth_header in response.request.headers.keys():
        del response.request.headers[auth_header]
        
    else:
        print("""WARNING! API authentication key has not been deleted from the response. 
              Please check the value passed to the `auth_header` parameter.""")

    with open(path, "wb") as f:
        pickle.dump(response, f)
        
        
# Pickle the response withou the "Authorization" field
pickle_response(response, "./yelp-api-laundromat-newark-response.pkl")

## 3. Save objects for valid responses
In this example we only have one valid response, but as you'll see the procedure below can easily be extended to multiple requests.

In [8]:
# Store requests parameter and the paths to the pickle file in a dict
test_params = params.copy()
test_params["base_url"] = base_url
test_params["pickle_path"] = "./yelp-api-laundromat-newark-response.pkl"

# Save list of dictionaries (containing pickled requests and their parameters) to JSON
save_requests_info([test_params], "./requests_info_tests.json")

Let's look at the file we just created.

In [9]:
!cat requests_info_tests.json 

[{"location": "Newark, NJ", "term": "laundromat", "limit": 5, "base_url": "https://api.yelp.com/v3/businesses/search", "pickle_path": "./yelp-api-laundromat-newark-response.pkl"}]

## 4. Errored requests
During the usage of `mock-request`, if you may want to set up a few error cases so that the user experience is more realistic (e.g if the request headers are missing). These errored responses are categorized by their _error codes_, and then saved as pickle files listed in a simple CSV.

In [10]:
# Parameters for few intentionally wrong request parameters
errored_requests_params = [
    # Wrong base URL
    {
        "pickle_file": "./yelp-api-wrong-base-url-response.pkl",
        "url": "https://api.yelp.com/v3/a_wrong_url",
        "headers": headers,
        "params": params
    },
    
    # Invalid API key
    {
        "pickle_file": "./yelp-api-invalid-key-response.pkl",
        "url": base_url,
        "params": params,
        "headers": {"Authorization": "Bearer " + "INVALID-API-KEY"}
    },
    
    # Wrong location parameter name
    {
        "pickle_file": "./yelp-api-wrong-location-parameter-response.pkl",
        "url": base_url,
        "headers": headers,
        "params": {
            "locationnnnnnn": "Newark, NJ",
            "term": "laundromat",
            "limit": 5
        }
    }
]

In [11]:
# Create lists of error codes and paths to pickled errored responses
error_codes = []
errored_response_paths = []

for erp in errored_requests_params:
    
    path = erp["pickle_file"]
        
    del(erp["pickle_file"])

    # Pickle errored response
    r = requests.get(**erp)
    
    pickle_response(r, path)

    # Save error code and path
    error_codes.append(r.status_code)
    errored_response_paths.append(path)
                    
        
# Create dataframe with error codes and paths
errors_tests = pd.DataFrame({
    "error_type": error_codes,
    "pickle_path": errored_response_paths
})

# Save dataframe to CSV file
errors_tests.to_csv("./errors_tests.csv", index=False)

The warning we see is from the example with no headers. So no worries, the API key has not been exposed! 

Let's explore the CSV file we created.

In [12]:
!cat ./errors_tests.csv

error_type,pickle_path
404,./yelp-api-wrong-base-url-response.pkl
400,./yelp-api-invalid-key-response.pkl
400,./yelp-api-wrong-location-parameter-response.pkl


## 5. Using these files to mock a request



In [13]:
from mock_request.requests import MockRequests

requests = MockRequests("./requests_info_tests.json", "./errors_tests.csv")

type(requests)

mock_request.requests.MockRequests

So now we're all set to mock the request! We write the exact same code as in cell `[6]`, but since the `requests` object is holding a `MockRequests` object (instead of the `requests` library), so we're calling a completely different function that does _not_ execute the request but rather loads the response from disk. Try disconnecting from the internet before you run the rest of this code!

In [14]:
# Execute mock request, using the same code as in [6]
mocked_response = requests.get(
    base_url,
    headers=headers,
    params=params
)

Are the response contents of the original response and the mocked response the same?

In [15]:
mocked_response.json() == response.json()

True

They are the same, so the mocked request worked!